In [1]:
using VCFTool

In [1]:
input_dir = "../input/"

output_dir = "../output/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_example_file_path = joinpath(input_dir, "example.vcf.gz")

"../input/example.vcf.gz"

In [6]:
vcf_738 = read_vcf(vcf_738_file_path)

vcf_example = read_vcf(vcf_example_file_path)

vcf_to_use = vcf_example

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
,String,Int64,String,String,String,Int64,String,String
1,1,10,1:10,A,T,100,PASS,NS=5;AC=0;AN=10;AF=0.000000
2,1,20,1:20,G,C,100,PASS,NS=0;AC=0;AN=0
3,1,30,1:30,C,A,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
4,1,40,1:40,A,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
5,1,10000,1:10000,G,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
6,1,20000,1:20000,T,A,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
7,4,5000,4:5000,A,T,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
8,4,6000,4:6000,C,T,100,PASS,NS=6;AC=11;AN=12;AF=0.916667
9,X,800,X:800,A,C,100,PASS,NS=6;AC=12;AN=12;AF=1.000000


## Tabix Regions

In [26]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

LoadError: UndefVarError: tabix_regions_from_file not defined

## Julia DataFrame Query 

In [6]:
regions = Dict([
        ("chr2", 240630710),
        ("chr7", 150999023),
        ("chr10", 99836239),
        ("chr10", 99851537),
        ("chr16", 88646828),
        ("chr19", 51354484),
        ("chr21", 36146408),
        ])

query_regions(
    regions, 
    vcf_to_use,
    output_dir,
)

chr10
99851537


0×5 DataFrame

chr16
88646828


0×5 DataFrame

chr19
51354484


0×5 DataFrame

chr7
150999023


0×5 DataFrame

chr2
240630710


0×5 DataFrame

chr21
36146408


0×5 DataFrame


Took 2 milliseconds.



## JuliaDB

In [3]:
using JuliaDB

vcf = loadtable(
    joinpath(input_dir, "738_variants.vcf"), 
    delim='\t',
    skiplines_begin=260,
    header_exists=false,
    colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
)

Table with 2847 rows, 10 columns:
Columns:
#   colname  type
───────────────────
1   CHROM    String
2   POS      String
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [9]:
loadndsparse(
    joinpath(input_dir, "738_variants.vcf"),
    delim='\t',
    skiplines_begin=260,
    header_exists=false, 
    colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
    indexcols=[:CHROM, :POS]
)

2-d NDSparse with 2847 values (8 field named tuples):
    Dimensions#  colname  type
──────────────────
1  CHROM    String
2  POS      String
    Values#   colname  type
───────────────────
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [ ]:
nd = ndsparse((x=x, y=y), (z=z,))

In [5]:
# chr1 1677720

vcf_table = table(vcf, pkey=[:POS])

Table with 2846 rows, 10 columns:
Columns:
#   colname  type
───────────────────
1   #CHROM   String
2   POS      Int64
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     Int64
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  Germ     String

In [31]:
using DataFrames

ENV["COLUMNS"]=100
ENV["LINES"]=200

DataFrame(vcf)

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
,String,Int64,String,String,String,Int64,String,String
1,chr1,10439,.,AC,A,72,PASS,CIGAR=1M1D;RU=C;REFREP=4;IDREP=3;MQ=9
2,chr1,13284,.,G,A,60,LowGQX;NoPassedVariantGTs,SNVHPOL=4;MQ=17
3,chr1,13868,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs,SNVHPOL=3;MQ=4
4,chr1,15274,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs,SNVHPOL=2;MQ=3
5,chr1,15903,.,G,GC,209,PASS,CIGAR=1M1I;RU=C;REFREP=2;IDREP=3;MQ=17
6,chr1,16683,.,T,G,0,LowGQX;NoPassedVariantGTs,SNVHPOL=7;MQ=10
7,chr1,16713,.,T,G,0,LowGQX;LowDepth;NoPassedVariantGTs,SNVHPOL=6;MQ=11
8,chr1,16731,.,C,G,0,LowGQX;NoPassedVariantGTs,SNVHPOL=8;MQ=16
9,chr1,16734,.,T,G,4,LowGQX;NoPassedVariantGTs,SNVHPOL=8;MQ=15


In [59]:
for item in vcf
    println(item)
end

(#CHROM = "1", POS = 10, ID = "1:10", REF = "A", ALT = "T", QUAL = 100, FILTER = "PASS", INFO = "NS=5;AC=0;AN=10;AF=0.000000", FORMAT = "GT", X1 = "0/0", X2 = "./.", X3 = "0/0", X4 = "0/0", X5 = "0/0", X6 = "0/0")
(#CHROM = "1", POS = 20, ID = "1:20", REF = "G", ALT = "C", QUAL = 100, FILTER = "PASS", INFO = "NS=0;AC=0;AN=0", FORMAT = "GT", X1 = "./.", X2 = "./.", X3 = "./.", X4 = "./.", X5 = "./.", X6 = "./.")
(#CHROM = "1", POS = 30, ID = "1:30", REF = "C", ALT = "A", QUAL = 100, FILTER = "PASS", INFO = "NS=6;AC=0;AN=12;AF=0.000000", FORMAT = "GT", X1 = "0/0", X2 = "0/0", X3 = "0/0", X4 = "0/0", X5 = "0/0", X6 = "0/0")
(#CHROM = "1", POS = 40, ID = "1:40", REF = "A", ALT = "C", QUAL = 100, FILTER = "PASS", INFO = "NS=6;AC=0;AN=12;AF=0.000000", FORMAT = "GT", X1 = "0/0", X2 = "0/0", X3 = "0/0", X4 = "0/0", X5 = "0/0", X6 = "0/0")
(#CHROM = "1", POS = 10000, ID = "1:10000", REF = "G", ALT = "C", QUAL = 100, FILTER = "PASS", INFO = "NS=6;AC=0;AN=12;AF=0.000000", FORMAT = "GT", X1 = "0/0

In [39]:
typeof(vcf)

IndexedTable{StructArrays.StructArray{NamedTuple{(Symbol("1_10_1:10_A_T_100_PASS_NS=5;AC=0;AN=10;AF=0.000000_GT_0/0_./._0/0_0/0_0/0_0/0"),),Tuple{String}},1,NamedTuple{(Symbol("1_10_1:10_A_T_100_PASS_NS=5;AC=0;AN=10;AF=0.000000_GT_0/0_./._0/0_0/0_0/0_0/0"),),Tuple{WeakRefStrings.StringArray{String,1}}},Int64}}

In [45]:
vcf[1][1]

"1\t20\t1:20\tG\tC\t100\tPASS\tNS=0;AC=0;AN=0\tGT\t./.\t./.\t./.\t./.\t./.\t./."